In [3]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
# Download data from open datasets
to_tensor = transforms.Compose([transforms.ToTensor()])
training_data = datasets.MNIST(root="data", train=True, download=True, transform=to_tensor)
test_data = datasets.MNIST(root="data", train=False, download=True, transform=to_tensor)

In [5]:
train_dataloader = DataLoader(training_data, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)

X, y = next(iter(test_dataloader))
print(f"Shape of X [N, C, H, W]: {X.shape}")
print(f"Shape of y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=28 * 28, out_features=512)
        # Output layer with 10 neurous for classification
        self.fc2 = nn.Linear(in_features=512, out_features=10)

    def forward(self, x):
        x = self.flatten(x) # Flatten the input tensor
        x = nn.functional.relu(self.fc1(x)) # ReLU activation after first layer
        x = self.fc2(x)
        return x
    
    
# Instantiate the model
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = NeuralNetwork().to(device)  # Move the model to the appropriate device
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*(correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.288735  [  128/60000]
loss: 0.361457  [12928/60000]
loss: 0.160356  [25728/60000]
loss: 0.111921  [38528/60000]
loss: 0.126698  [51328/60000]
Test Error: 
 Accuracy: 95.6%, Avg loss: 0.147090

Epoch 2
-------------------------------
loss: 0.100620  [  128/60000]
loss: 0.114104  [12928/60000]
loss: 0.054357  [25728/60000]
loss: 0.158951  [38528/60000]
loss: 0.086845  [51328/60000]
Test Error: 
 Accuracy: 97.1%, Avg loss: 0.099007

Epoch 3
-------------------------------
loss: 0.074525  [  128/60000]
loss: 0.116135  [12928/60000]
loss: 0.090858  [25728/60000]
loss: 0.044091  [38528/60000]
loss: 0.087000  [51328/60000]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.078157

Epoch 4
-------------------------------
loss: 0.054401  [  128/60000]
loss: 0.045387  [12928/60000]
loss: 0.091889  [25728/60000]
loss: 0.058447  [38528/60000]
loss: 0.010375  [51328/60000]
Test Error: 
 Accuracy: 98.0%, Avg loss: 0.066888

Epoch 5
----------------------------

In [ ]:
torch.save(model.state_dict(), "mnist_model.pth")
print("Saved PyTorch Model State to mnist_model.pth")